In [1]:
# 📦 Install and import dependencies
!pip install simpy --quiet

import simpy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ==== CONFIGURATION ====
TOTAL_NODES = 20
SELFISH_NODES = 5
SIM_SLOTS_PER_PHASE = 3000
CW_MIN = 16
CW_MAX = 1024
SELFISH_CW_MIN = 4
SELFISH_CW_MAX = 32
SLOT_DURATION_SEC = 0.001
PACKET_SIZE_BYTES = 1024

# ==== METRIC HELPERS ====
def jfi(values):
    values = np.array(values)
    if np.sum(values) == 0: return 0
    return (np.sum(values)**2) / (len(values) * np.sum(values**2))

def throughput_mbps(packets, duration_slots):
    total_bits = packets * PACKET_SIZE_BYTES * 8
    return total_bits / (duration_slots * SLOT_DURATION_SEC * 1e6)

# ==== SHARED CHANNEL ====
class SharedChannel:
    def __init__(self, env):
        self.env = env
        self.transmitting_nodes = []
        self.busy = False

    def occupy(self, node):
        self.transmitting_nodes.append(node)
        self.busy = True

    def release(self):
        self.transmitting_nodes.clear()
        self.busy = False

# ==== NODE CLASS ====
class Node:
    def __init__(self, env, node_id, is_selfish, channel):
        self.env = env
        self.id = node_id
        self.is_selfish = is_selfish
        self.channel = channel

        self.cw_min = SELFISH_CW_MIN if is_selfish else CW_MIN
        self.cw_max = SELFISH_CW_MAX if is_selfish else CW_MAX
        self.cw = self.cw_min

        self.packets_sent = 0
        self.collisions = 0
        self.attempts = 0
        self.action = env.process(self.run())

    def run(self):
        while True:
            self.cw = max(self.cw_min, min(self.cw, self.cw_max))
            backoff_timer = random.randint(0, self.cw)
            while backoff_timer > 0:
                yield self.env.timeout(1)
                if not self.channel.busy:
                    backoff_timer -= 1

            self.attempts += 1
            self.channel.occupy(self)
            yield self.env.timeout(1)

            if len(self.channel.transmitting_nodes) == 1:
                self.packets_sent += 1
                self.cw = self.cw_min
            else:
                self.collisions += 1
                self.cw = min(int(self.cw * 2), self.cw_max)

            self.channel.release()
            yield self.env.timeout(1)

# ==== SIMULATION RUNNER ====
def run_simulation(is_selfish_phase, cw_override=None, total_nodes=None, selfish_nodes=None):
    if total_nodes is None:
        total_nodes = TOTAL_NODES  # fallback to default
    if selfish_nodes is None:
        selfish_nodes = SELFISH_NODES  # fallback to default

    env = simpy.Environment()
    channel = SharedChannel(env)
    nodes = []
    selfish_ids = random.sample(range(total_nodes), selfish_nodes) if is_selfish_phase else []

    for i in range(total_nodes):
        is_selfish = i in selfish_ids
        node = Node(env, i, is_selfish, channel)
        if cw_override and i in cw_override:
            node.cw_min = cw_override[i]
            node.cw = cw_override[i]
        nodes.append(node)

    env.run(until=SIM_SLOTS_PER_PHASE)
    return nodes, selfish_ids


# ==== METRIC CALCULATOR ====
def get_metrics(nodes, label):
    throughput = [n.packets_sent for n in nodes]
    collisions = [n.collisions for n in nodes]
    jfi_score = jfi(throughput)
    total_packets = sum(throughput)
    total_collisions = sum(collisions)
    total_attempts = sum([n.attempts for n in nodes])
    throughput_val = throughput_mbps(total_packets, SIM_SLOTS_PER_PHASE)
    collision_rate = total_collisions / total_attempts if total_attempts else 0

    return {
        "Phase": label,
        "Throughput (Mbps)": round(throughput_val, 6),
        "JFI": round(jfi_score, 4),
        "Collision Rate": round(collision_rate, 4)
    }

# ==== PHASE 1: BEFORE ATTACK ====
nodes_before, _ = run_simulation(False)
metrics_before = get_metrics(nodes_before, "Before Attack")

# ==== PHASE 2: AFTER ATTACK ====
nodes_after, selfish_ids = run_simulation(True)
metrics_after = get_metrics(nodes_after, "After Attack")

# ==== DETECTION ====
def detect_selfish_nodes(nodes, selfish_ids, z_thresh=1.0):
    data = []
    for node in nodes:
        sr = node.packets_sent / node.attempts if node.attempts else 0
        data.append({
            'Node': node.id,
            'Selfish': node.id in selfish_ids,
            'Throughput': node.packets_sent,
            'Attempts': node.attempts,
            'SuccessRatio': sr
        })
    df = pd.DataFrame(data)
    for col in ["Throughput", "Attempts", "SuccessRatio"]:
        df[f"{col}_z"] = (df[col] - df[col].mean()) / df[col].std()
    def is_suspicious(row):
        return sum([
            row["Throughput_z"] > z_thresh,
            row["Attempts_z"] > z_thresh,
            row["SuccessRatio_z"] > z_thresh
        ]) >= 2
    df["Detected_Selfish"] = df.apply(is_suspicious, axis=1)
    return df

df_detected = detect_selfish_nodes(nodes_after, selfish_ids)



# === Detection Reporting ===
true_selfish_nodes = df_detected[df_detected["Selfish"] == True]["Node"].tolist()
detected_selfish_nodes = df_detected[df_detected["Detected_Selfish"] == True]["Node"].tolist()

true_positives = df_detected[(df_detected["Selfish"] == True) & (df_detected["Detected_Selfish"] == True)]
false_positives = df_detected[(df_detected["Selfish"] == False) & (df_detected["Detected_Selfish"] == True)]
false_negatives = df_detected[(df_detected["Selfish"] == True) & (df_detected["Detected_Selfish"] == False)]
true_negatives = df_detected[(df_detected["Selfish"] == False) & (df_detected["Detected_Selfish"] == False)]

accuracy = (len(true_positives) + len(true_negatives)) / len(df_detected)

print("\n📌 Detection Phase Summary")
print("✅ Actual Selfish Nodes:    ", sorted(true_selfish_nodes))
print("🕵️‍♂️ Detected Selfish Nodes:", sorted(detected_selfish_nodes))
print("❌ False Positives:         ", sorted(false_positives['Node'].tolist()))
print("❌ False Negatives:         ", sorted(false_negatives['Node'].tolist()))
print(f"🎯 Detection Accuracy:      {accuracy*100:.2f}%")
print("*" * 60)

# ==== STATIC MITIGATION ====
def run_static_mitigation(df_detected, alpha=2.0):
    cw_override = {}
    for _, row in df_detected.iterrows():
        if row["Detected_Selfish"]:
            cw_override[row["Node"]] = int(CW_MIN * (1 + alpha))
    return run_simulation(True, cw_override)

nodes_static, _ = run_static_mitigation(df_detected)
metrics_static = get_metrics(nodes_static, "After Static Mitigation")

# ==== BSCR 2.0 MITIGATION ====
def run_bscr_mitigation_with_decay(df_detected, alpha=0.4, beta=0.4, gamma=0.2,
                                    delta=2.0, threshold=0.3, decay_factor=0.9):
    env = simpy.Environment()
    channel = SharedChannel(env)
    nodes = []

    cw_override = {}

    # Step 1: Calculate behavior scores and initial CW penalties
    for _, row in df_detected.iterrows():
        node_id = row["Node"]
        score = alpha * row["Throughput_z"] + beta * row["Attempts_z"] + gamma * row["SuccessRatio_z"]
        if row["Detected_Selfish"]:
            if score > threshold:
                penalized_cw = int(CW_MIN * (1 + delta * score))
                cw_override[node_id] = penalized_cw
            else:
                # Recovery case: gradually decay CW if below threshold
                prev_cw = cw_override.get(node_id, CW_MIN)
                cw_override[node_id] = max(CW_MIN, round(prev_cw * decay_factor))

    # Step 2: Create nodes and apply CWs
    selfish_ids = df_detected[df_detected["Selfish"] == True]["Node"].tolist()
    for i in range(TOTAL_NODES):
        is_selfish = i in selfish_ids
        node = Node(env, i, is_selfish, channel)
        if i in cw_override:
            node.cw_min = cw_override[i]
            node.cw = cw_override[i]
        nodes.append(node)

    env.run(until=SIM_SLOTS_PER_PHASE)
    return nodes

nodes_bscr_decay = run_bscr_mitigation_with_decay(df_detected)
metrics_bscr_decay = get_metrics(nodes_bscr_decay, "After BSCR 2.0 + Decay")

def run_bscr_mitigation_fairness_aware(df_detected, alpha=0.4, beta=0.4, gamma=0.2,
                                        delta=2.0, threshold=0.3):
    env = simpy.Environment()
    channel = SharedChannel(env)
    nodes = []

    selfish_ids = df_detected[df_detected["Selfish"] == True]["Node"].tolist()

    # --- Estimate current JFI to adapt decay rate ---
    all_scores = []
    for _, row in df_detected.iterrows():
        score = alpha * row["Throughput_z"] + beta * row["Attempts_z"] + gamma * row["SuccessRatio_z"]
        all_scores.append(score)

    avg_score = np.mean(all_scores)
    std_score = np.std(all_scores)
    jfi_estimate = 1 - (std_score / (avg_score + 1e-5))

    if jfi_estimate < 0.7:
        decay_factor = 0.90
    elif jfi_estimate < 0.9:
        decay_factor = 0.95
    else:
        decay_factor = 0.98

    cw_override = {}
    for _, row in df_detected.iterrows():
        node_id = row["Node"]
        score = alpha * row["Throughput_z"] + beta * row["Attempts_z"] + gamma * row["SuccessRatio_z"]
        if row["Detected_Selfish"]:
            if score > threshold:
                cw_override[node_id] = int(CW_MIN * (1 + delta * score))
            else:
                prev_cw = cw_override.get(node_id, CW_MIN)
                cw_override[node_id] = max(CW_MIN, round(prev_cw * decay_factor))

    for i in range(TOTAL_NODES):
        is_selfish = i in selfish_ids
        node = Node(env, i, is_selfish, channel)
        if i in cw_override:
            node.cw_min = cw_override[i]
            node.cw = cw_override[i]
        nodes.append(node)

    env.run(until=SIM_SLOTS_PER_PHASE)
    return nodes
nodes_bscr_fair = run_bscr_mitigation_fairness_aware(df_detected)
metrics_bscr_fair = get_metrics(nodes_bscr_fair, "BSCR 2.0 + Fairness-Aware Decay")


#STRIKE-2
def run_bscr_strike2_mitigation(df_detected, alpha=0.4, beta=0.4, gamma=0.2,
                                 delta=2.0, threshold=0.3, decay_factor=0.9,
                                 strike_penalty=1.5):
    env = simpy.Environment()
    channel = SharedChannel(env)
    nodes = []

    selfish_ids = df_detected[df_detected["Selfish"] == True]["Node"].tolist()
    offense_count = {}
    cw_override = {}

    for _, row in df_detected.iterrows():
        node_id = row["Node"]
        score = alpha * row["Throughput_z"] + beta * row["Attempts_z"] + gamma * row["SuccessRatio_z"]
        detected = row["Detected_Selfish"]

        if detected:
            offense_count[node_id] = offense_count.get(node_id, 0) + 1
            strike_multiplier = strike_penalty if offense_count[node_id] >= 2 else 1.0
            penalized_cw = int(CW_MIN * (1 + delta * score * strike_multiplier))
            cw_override[node_id] = penalized_cw
        else:
            if node_id in cw_override:
                prev_cw = cw_override[node_id]
                cw_override[node_id] = max(CW_MIN, round(prev_cw * decay_factor))

    for i in range(TOTAL_NODES):
        is_selfish = i in selfish_ids
        node = Node(env, i, is_selfish, channel)
        if i in cw_override:
            node.cw_min = cw_override[i]
            node.cw = cw_override[i]
        nodes.append(node)

    env.run(until=SIM_SLOTS_PER_PHASE)
    return nodes

nodes_strike2 = run_bscr_strike2_mitigation(df_detected)
metrics_strike2 = get_metrics(nodes_strike2, "BSCR 2.0 + Decay + Strike-2")
# 📦 Ensure required imports
import random
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ✅ Multi-Round BSCR + Decay + Strike-2 Adaptive Mitigation
def run_multi_round_mitigation(rounds=3, alpha=0.4, beta=0.4, gamma=0.2,
                                delta=2.0, threshold=0.3, decay_factor=0.9,
                                strike_penalty=1.5):
    selfish_ids = random.sample(range(TOTAL_NODES), SELFISH_NODES)
    offense_count = {}
    cw_override = {}
    round_metrics = []

    # First attack round (no CW override)
    def run_initial_sim():
        env = simpy.Environment()
        channel = SharedChannel(env)
        nodes = []
        for i in range(TOTAL_NODES):
            is_selfish = i in selfish_ids
            node = Node(env, i, is_selfish, channel)
            nodes.append(node)
        env.run(until=SIM_SLOTS_PER_PHASE)
        return nodes

    current_nodes = run_initial_sim()

    for round_idx in range(1, rounds + 1):
        print(f"\n🔁 Round {round_idx} Detection + Mitigation")

        # === Detection Phase
        df = detect_selfish_nodes(current_nodes, selfish_ids)

        # === CW Adaptation
        new_cw_override = {}
        for _, row in df.iterrows():
            node_id = row["Node"]
            score = alpha * row["Throughput_z"] + beta * row["Attempts_z"] + gamma * row["SuccessRatio_z"]
            detected = row["Detected_Selfish"]

            if detected:
                offense_count[node_id] = offense_count.get(node_id, 0) + 1
                multiplier = strike_penalty if offense_count[node_id] >= 2 else 1.0
                penalized_cw = int(CW_MIN * (1 + delta * score * multiplier))
                new_cw_override[node_id] = penalized_cw
            else:
                if node_id in cw_override:
                    prev_cw = cw_override[node_id]
                    new_cw_override[node_id] = max(CW_MIN, round(prev_cw * decay_factor))

        cw_override = new_cw_override.copy()

        # === Run Mitigation Round with Updated CWs
        def run_with_cw():
            env = simpy.Environment()
            channel = SharedChannel(env)
            nodes = []
            for i in range(TOTAL_NODES):
                is_selfish = i in selfish_ids
                node = Node(env, i, is_selfish, channel)
                if i in cw_override:
                    node.cw_min = cw_override[i]
                    node.cw = cw_override[i]
                nodes.append(node)
            env.run(until=SIM_SLOTS_PER_PHASE)
            return nodes

        current_nodes = run_with_cw()
        metrics = get_metrics(current_nodes, f"Round-{round_idx}")
        round_metrics.append(metrics)

    return pd.DataFrame(round_metrics)

# 🔁 Run and Display Results
df_multi_round = run_multi_round_mitigation(rounds=3)





# ==== RESULT TABLE ====
summary_df_strike2 = pd.DataFrame([
    metrics_before,
    metrics_after,
    metrics_static,
    #metrics_bscr,
    #metrics_bscr_decay,
    metrics_bscr_fair,
    #metrics_strike2
])

print(summary_df_strike2)


# === Combine All Methods + Multi-Round Final Round into Summary Table ===

# Use last round of multi-round as final metric
metrics_multi_round_final = {
    "Phase": "Multi-Round Final",
    "Throughput (Mbps)": df_multi_round.iloc[-1]["Throughput (Mbps)"],
    "JFI": df_multi_round.iloc[-1]["JFI"],
    "Collision Rate": df_multi_round.iloc[-1]["Collision Rate"]
}

# Combine with existing metrics (assumed already computed)
summary_df_all = pd.DataFrame([
    metrics_before,
    metrics_after,
    metrics_static,
    #metrics_bscr,
    #metrics_bscr_decay,
    metrics_bscr_fair,
    #metrics_strike2,
    metrics_multi_round_final
])
# 📦 Ensure imports (if not already)
import pandas as pd
import matplotlib.pyplot as plt

# ==== Configurations for Experiments ====

scalability_node_sizes = [20, 50, 100]
severity_settings = {
    "Mild Attack (CW 1-8)": (1, 8),
    "Aggressive Attack (CW 1-4)": (1, 4)
}

selfish_node_ratio = 0.2  # 20% selfish nodes

scalability_results = []
severity_results = []

# ==== Scalability Experiment ====
for total_nodes in scalability_node_sizes:
    selfish_nodes = int(total_nodes * selfish_node_ratio)
    print(f"Running Scalability Simulation for {total_nodes} nodes...")

    nodes, _ = run_simulation(
        is_selfish_phase=True,
        total_nodes=total_nodes,
        selfish_nodes=selfish_nodes
    )
    metrics = get_metrics(nodes, f"{total_nodes} Nodes")
    scalability_results.append(metrics)

# ==== Attack Severity Experiment ====
for label, (cw_min, cw_max) in severity_settings.items():
    print(f"Running Attack Severity Simulation: {label}...")
    SELFISH_CW_MIN = cw_min
    SELFISH_CW_MAX = cw_max

    nodes, _ = run_simulation(
        is_selfish_phase=True,
        total_nodes=50,
        selfish_nodes=10
    )
    metrics = get_metrics(nodes, label)
    severity_results.append(metrics)

# ==== Collect Results ====
scalability_df = pd.DataFrame(scalability_results)
severity_df = pd.DataFrame(severity_results)

# ==== Display Results ====
print("\n📈 Scalability Results:")
print(scalability_df)

print("\n🔥 Attack Severity Results:")
print(severity_df)


#summary_df_all

# ==== OPTIONAL: BAR PLOT ====
#summary_df.plot(x='Phase', kind='bar', figsize=(10,6))
#plt.title("DARMAC Performance Comparison")
#plt.xticks(rotation=15)
#plt.grid(True, linestyle='--', alpha=0.6)
#plt.tight_layout()
#plt.show()



📌 Detection Phase Summary
✅ Actual Selfish Nodes:     [3, 6, 7, 12, 19]
🕵️‍♂️ Detected Selfish Nodes: [3, 6, 7, 12, 19]
❌ False Positives:          []
❌ False Negatives:          []
🎯 Detection Accuracy:      100.00%
************************************************************

🔁 Round 1 Detection + Mitigation

🔁 Round 2 Detection + Mitigation

🔁 Round 3 Detection + Mitigation
                             Phase  Throughput (Mbps)     JFI  Collision Rate
0                    Before Attack           5.469525  0.9929          0.0982
1                     After Attack           6.059349  0.6218          0.2017
2          After Static Mitigation           5.778091  0.5979          0.1999
3  BSCR 2.0 + Fairness-Aware Decay           5.114539  0.8720          0.0895
Running Scalability Simulation for 20 nodes...
Running Scalability Simulation for 50 nodes...
Running Scalability Simulation for 100 nodes...
Running Attack Severity Simulation: Mild Attack (CW 1-8)...
Running Attack Severity Sim